In [1]:
import pandas as pd
import re

# Load the CSV files
main_df = pd.read_csv('./dataset/train_new.csv')
ffpp_df = pd.read_csv('./dataset/A-FF++.csv')
'''
data = {
    "Image Path": ["/shared/rc/defake/FaceForensics++_All/FaceForensics++/fake/DeepFakes/test/altered/379_158/191.jpg"]
}

# Create the DataFrame
main_df = pd.DataFrame(data)

data = {
    "path": ["FaceForensics++/manipulated_sequences/DeepFakes/raw/face_images/379_158/frame151.png"]
}

# Create the DataFrame
ffpp_df = pd.DataFrame(data)
'''
# Function to extract parts from `main.csv` paths
def extract_main_parts(path):
    #/shared/rc/defake/FaceForensics++_All/FaceForensics++/real/youtube/train/907.mp4/0-211.jpeg
    if 'real' in path:
        match = re.search(r'/([^/]+)/([^/]+)/([^/]+)/([^/]+)\.mp4/(\d+)-(\d+)\.jpeg', path)
        if match:
            return match.group(2), match.group(4), match.group(6)
        return None, None, None
        
    elif 'DeepFakes' in path:
        #/shared/rc/defake/FaceForensics++_All/FaceForensics++/fake/DeepFakes/val/altered/471_455/101.jpg
        match = re.search(r'/([^/]+)/[^/]+/([^/]+)/([^/]+)/(\d+)\.jpg', path)
        if match:
            return match.group(1), match.group(3), match.group(4)
        return None, None, None
        
    else:
        #/shared/rc/defake/FaceForensics++_All/FaceForensics++/fake/Face2Face/train/624_570.mp4/0-221.jpeg
        match = re.search(r'/([^/]+)/[^/]+/([^/]+)/([^/]+)/([^/]+)\.mp4/(\d+)-(\d+)\.jpeg', path)
        if match:
            return match.group(2), match.group(4), match.group(6)
        return None, None, None

# Function to extract parts from `ff++.csv` paths
def extract_ffpp_parts(path):
    #FaceForensics++/original_sequences/youtube/raw/face_images/999/frame131.png
    if 'original_sequences' in path:
        match = re.search(r'([^/]+)/[^/]+/([^/]+)/[^/]+/[^/]+/([^/]+)/frame(\d+)\.png', path)
        if match:
            return match.group(2), match.group(3), match.group(4)
        return None, None, None

    else:
        #FaceForensics++/manipulated_sequences/Deepfakes/raw/face_images/836_950/frame251.png
        match = re.search(r'([^/]+)/[^/]+/([^/]+)/[^/]+/[^/]+/([^/]+)/frame(\d+)\.png', path)
        if match:
            return match.group(2), match.group(3), match.group(4)  # Adjusted to capture the correct order
        return None, None, None

# Add extracted parts as new columns in both DataFrames

main_df[['group', 'video_id', 'frame']] = main_df['Image Path'].apply(
    lambda x: pd.Series(extract_main_parts(x))
)

ffpp_df[['group', 'video_id', 'frame']] = ffpp_df['path'].apply(
    lambda x: pd.Series(extract_ffpp_parts(x))
)
ffpp_dedup = ffpp_df.drop_duplicates(subset=['group', 'video_id'])

merged_df = pd.merge(
    main_df,
    ffpp_dedup[['group', 'video_id','black_hair','blond_hair','gray_hair','no_eyewear']],
    on=['group', 'video_id'],
    how='left'  # Keep all rows from main_df and add matching data from ffpp_df
)

#Save the updated main_df back to a new CSV
merged_df.to_csv('train_final.csv', index=False)

In [2]:
deepfakes_df = main_df[main_df['group'] == 'DeepFakes']



In [3]:
deepfakes_df

,Image Path,Uncertainty Score Gender,Uncertainty Score Age,Uncertainty Score Race,Ground Truth Gender,Ground Truth Age,Ground Truth Race,Intersection,Target,Fake classification,group,video_id,frame
11879,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200300,0.203947,0.203622,0,0,1,5,1,1,DeepFakes,931_936,191
11880,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.223067,0.641747,0.238041,0,3,1,5,1,1,DeepFakes,803_017,291
11889,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.203442,0.254839,0.200609,0,0,0,4,1,1,DeepFakes,471_455,101
11895,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.280204,0.272480,0.206260,0,3,1,5,1,1,DeepFakes,788_710,81
11898,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200170,0.200311,0.208835,0,0,1,5,1,1,DeepFakes,075_977,271
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56684,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.209660,0.420040,0.254155,1,1,1,1,1,1,DeepFakes,919_015,161
56686,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.259873,0.616962,0.454046,1,3,1,1,1,1,DeepFakes,333_377,191
56688,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.208752,0.211526,0.655986,0,0,0,4,1,1,DeepFakes,752_751,291
56699,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200087,0.200186,0.209819,0,0,0,4,1,1,DeepFakes,759_755,101


In [4]:
main_df

,Image Path,Uncertainty Score Gender,Uncertainty Score Age,Uncertainty Score Race,Ground Truth Gender,Ground Truth Age,Ground Truth Race,Intersection,Target,Fake classification,group,video_id,frame
0,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.201479,0.447477,0.201614,1,2,1,1,0,0,youtube,002,215
1,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200740,0.200522,0.210722,0,0,1,5,0,0,youtube,698,176
2,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.203873,0.535472,0.217305,1,3,1,1,0,0,youtube,761,109
3,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200120,0.200078,0.241493,0,0,1,5,0,0,youtube,038,111
4,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.202236,0.247109,0.283761,1,1,3,3,0,0,youtube,100,141
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56700,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.205472,0.528283,0.251096,1,0,1,1,1,3,FaceSwap,879_963,571
56701,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.207070,0.202177,0.203466,1,0,0,0,1,4,NeuralTextures,584_823,271
56702,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.201134,0.202542,0.423039,0,0,1,5,1,1,DeepFakes,992_980,121
56703,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200100,0.200510,0.202008,0,0,1,5,1,3,FaceSwap,529_633,171


In [5]:
deepfakes_dff = ffpp_dedup[ffpp_dedup['video_id'] == '992_980']
deepfakes_dff

,Unnamed: 0,path,label,male,young,middle_aged,senior,asian,white,black,...,heavy_makeup,wearing_hat,wearing_necktie,wearing_lipstick,no_eyewear,eyeglasses,attractive,group,video_id,frame
29634,29634,FaceForensics++/manipulated_sequences/NeuralTe...,1,-1,1,-1,-1,-1,1,-1,...,0,-1,0,1,1,-1,1,NeuralTextures,992_980,101
59439,59439,FaceForensics++/manipulated_sequences/FaceSwap...,1,-1,0,-1,-1,-1,1,-1,...,0,-1,-1,0,1,-1,0,FaceSwap,992_980,101
89291,89291,FaceForensics++/manipulated_sequences/FaceShif...,1,-1,0,-1,-1,0,0,-1,...,0,-1,-1,1,1,-1,0,FaceShifter,992_980,101
119006,119006,FaceForensics++/manipulated_sequences/DeepFake...,1,-1,1,-1,-1,-1,1,-1,...,0,-1,-1,1,1,-1,0,DeepFakes,992_980,101
148885,148885,FaceForensics++/manipulated_sequences/Face2Fac...,1,-1,1,-1,-1,-1,1,-1,...,1,-1,-1,1,1,-1,1,Face2Face,992_980,101


In [6]:
merged_df

,Image Path,Uncertainty Score Gender,Uncertainty Score Age,Uncertainty Score Race,Ground Truth Gender,Ground Truth Age,Ground Truth Race,Intersection,Target,Fake classification,group,video_id,frame,black_hair,blond_hair,gray_hair,no_eyewear
0,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.201479,0.447477,0.201614,1,2,1,1,0,0,youtube,002,215,-1,-1,0,-1
1,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200740,0.200522,0.210722,0,0,1,5,0,0,youtube,698,176,0,-1,-1,1
2,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.203873,0.535472,0.217305,1,3,1,1,0,0,youtube,761,109,0,-1,-1,1
3,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200120,0.200078,0.241493,0,0,1,5,0,0,youtube,038,111,0,-1,-1,1
4,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.202236,0.247109,0.283761,1,1,3,3,0,0,youtube,100,141,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56700,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.205472,0.528283,0.251096,1,0,1,1,1,3,FaceSwap,879_963,571,0,-1,0,1
56701,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.207070,0.202177,0.203466,1,0,0,0,1,4,NeuralTextures,584_823,271,1,-1,-1,1
56702,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.201134,0.202542,0.423039,0,0,1,5,1,1,DeepFakes,992_980,121,0,0,-1,1
56703,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200100,0.200510,0.202008,0,0,1,5,1,3,FaceSwap,529_633,171,0,0,-1,1


In [7]:
merged_dff = merged_df[merged_df['video_id'] == '992_980']
merged_dff

,Image Path,Uncertainty Score Gender,Uncertainty Score Age,Uncertainty Score Race,Ground Truth Gender,Ground Truth Age,Ground Truth Race,Intersection,Target,Fake classification,group,video_id,frame,black_hair,blond_hair,gray_hair,no_eyewear
11954,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.201319,0.203137,0.413089,0,0,1,5,1,2,Face2Face,992_980,111,0,-1,-1,1
12034,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200580,0.200606,0.271537,0,0,1,5,1,1,DeepFakes,992_980,151,0,0,-1,1
12463,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200324,0.200415,0.247118,0,0,1,5,1,4,NeuralTextures,992_980,221,0,-1,-1,1
13315,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200486,0.200483,0.394877,0,0,1,5,1,4,NeuralTextures,992_980,191,0,-1,-1,1
13683,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200401,0.201104,0.289254,0,0,1,5,1,2,Face2Face,992_980,271,0,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53830,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.203272,0.206708,0.553097,0,0,1,5,1,2,Face2Face,992_980,241,0,-1,-1,1
53983,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200139,0.200087,0.232040,0,0,1,5,1,4,NeuralTextures,992_980,231,0,-1,-1,1
54315,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200406,0.201118,0.263210,0,0,1,5,1,1,DeepFakes,992_980,116,0,0,-1,1
56455,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200391,0.200291,0.229657,0,0,1,5,1,4,NeuralTextures,992_980,261,0,-1,-1,1


In [8]:
def get_hair_color(row):
    if row['gray_hair'] == 1:
        return 1
    elif row['black_hair'] == 1:
        return 2
    elif row['blond_hair'] == 1:
        return 3
    else:
        return 0  # For cases where no definitive conclusion can be made

# Only process rows with matches
merged_df['hair_color'] = merged_df.apply(
    lambda row: get_hair_color(row) ,
    axis=1
)
merged_df.to_csv('train_final.csv', index=False)

In [9]:
merged_dff = merged_df[merged_df['group'] == 'DeepFakes']
merged_dff

,Image Path,Uncertainty Score Gender,Uncertainty Score Age,Uncertainty Score Race,Ground Truth Gender,Ground Truth Age,Ground Truth Race,Intersection,Target,Fake classification,group,video_id,frame,black_hair,blond_hair,gray_hair,no_eyewear,hair_color
11879,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200300,0.203947,0.203622,0,0,1,5,1,1,DeepFakes,931_936,191,-1,0,-1,0,0
11880,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.223067,0.641747,0.238041,0,3,1,5,1,1,DeepFakes,803_017,291,0,-1,-1,0,0
11889,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.203442,0.254839,0.200609,0,0,0,4,1,1,DeepFakes,471_455,101,0,-1,0,0,0
11895,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.280204,0.272480,0.206260,0,3,1,5,1,1,DeepFakes,788_710,81,0,0,0,1,0
11898,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200170,0.200311,0.208835,0,0,1,5,1,1,DeepFakes,075_977,271,0,-1,-1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56684,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.209660,0.420040,0.254155,1,1,1,1,1,1,DeepFakes,919_015,161,0,-1,0,1,0
56686,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.259873,0.616962,0.454046,1,3,1,1,1,1,DeepFakes,333_377,191,0,-1,0,1,0
56688,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.208752,0.211526,0.655986,0,0,0,4,1,1,DeepFakes,752_751,291,1,-1,-1,1,2
56699,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200087,0.200186,0.209819,0,0,0,4,1,1,DeepFakes,759_755,101,0,-1,0,1,0


In [10]:
merged_df

,Image Path,Uncertainty Score Gender,Uncertainty Score Age,Uncertainty Score Race,Ground Truth Gender,Ground Truth Age,Ground Truth Race,Intersection,Target,Fake classification,group,video_id,frame,black_hair,blond_hair,gray_hair,no_eyewear,hair_color
0,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.201479,0.447477,0.201614,1,2,1,1,0,0,youtube,002,215,-1,-1,0,-1,0
1,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200740,0.200522,0.210722,0,0,1,5,0,0,youtube,698,176,0,-1,-1,1,0
2,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.203873,0.535472,0.217305,1,3,1,1,0,0,youtube,761,109,0,-1,-1,1,0
3,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200120,0.200078,0.241493,0,0,1,5,0,0,youtube,038,111,0,-1,-1,1,0
4,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.202236,0.247109,0.283761,1,1,3,3,0,0,youtube,100,141,1,-1,-1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56700,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.205472,0.528283,0.251096,1,0,1,1,1,3,FaceSwap,879_963,571,0,-1,0,1,0
56701,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.207070,0.202177,0.203466,1,0,0,0,1,4,NeuralTextures,584_823,271,1,-1,-1,1,2
56702,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.201134,0.202542,0.423039,0,0,1,5,1,1,DeepFakes,992_980,121,0,0,-1,1,0
56703,/shared/rc/defake/FaceForensics++_All/FaceFore...,0.200100,0.200510,0.202008,0,0,1,5,1,3,FaceSwap,529_633,171,0,0,-1,1,0
